In [ ]:
from alarms.costs import ItemFinderSecop2, Item
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
secop_union = pd.read_pickle('../data/secop_union_all.pickle')

In [ ]:
secop_2_covid = secop_union.loc[lambda x: (x.source == 'secop_2') & (x.is_covid)]
urls = secop_2_covid.url

In [ ]:
urls.shape

In [ ]:
import time
import pickle
import numpy as np
from tqdm import tqdm

In [ ]:
item_lists = []
finder = ItemFinderSecop2()

for url in tqdm(urls):
    try:
        items = finder.find_items(url)
        item_lists.append(items)
    except Exception:
        item_lists.append(None)
    
    time.sleep(np.random.uniform(0,0.5))

In [ ]:
# item_lists[1000]

In [ ]:
len(item_lists)

In [ ]:
with open('../data/item_lists.pkl', 'wb') as f:
    pickle.dump(item_lists, f)

In [ ]:
secop_2_items = secop_2_covid[['id_contrato']].copy()

In [ ]:
secop_2_items['item_list'] = item_lists
secop_2_items = secop_2_items.explode('item_list')

In [ ]:
default_item = Item(None, None,None,None, None)
secop_2_items['item_list'] = secop_2_items.item_list.fillna(default_item)

item_cols = list(map(lambda c: 'item_{}'.format(c), ['code', 'description', 'quantity', 'unit', 'price']))
secop_2_items[item_cols] = secop_2_items.apply(lambda x: x.item_list.as_dict(), axis=1, result_type="expand")

In [ ]:
secop_2_items.drop('item_list', axis=1).to_pickle('../data/secop_2_covid_items.pkl')
secop_2_items.drop('item_list', axis=1).to_csv('../data/secop_2_covid_items.csv')

In [ ]:
secop_2_items.drop('item_list', axis=1).to_feather('../data/secop_2_covid_items.feather')

In [ ]:
secop_2_items.head()

## exploration

In [ ]:
item_counts = secop_2_items.groupby('item_description').agg(n = ('item_description', 'count')).reset_index()

In [ ]:
shared = item_counts.loc[lambda x: (x.n>1)].sort_values('n', ascending=False)

In [ ]:
secop_2_items['item_description'] = secop_2_items['item_description'].fillna('NA')

In [ ]:
tecnicos = secop_2_items.loc[lambda x: x.item_description.str.contains('TECNICO EN AUXILIAR DE ENFERMERIA')]

In [ ]:
tecnicos.groupby('id_contrato').agg(n=('id_contrato', 'count'))

In [ ]:
secop_2_covid.loc[lambda x: x.id_contrato=='CO1.PCCNTR.1577446'].url.iloc[0]

In [ ]:
shared.loc[lambda x: ~x.item_description.str.lower().str.contains('tecnico|prestaci|municipio|valor|honorarios')].tail(50)

In [ ]:
shared.iloc[100]

In [ ]:
secop_2_covid.groupby('tipo_de_contrato').agg(n=('tipo_de_contrato', 'count')).reset_index()

In [ ]:
secop_2_covid.loc[lambda x: x.tipo_de_contrato=='Arrendamiento de inmuebles'].iloc[0]

In [ ]:
secop_2_covid.loc[lambda x: x.tipo_de_contrato=='Suministros'].iloc[3]

In [ ]:
# secop_1_covid.detalle_del_objeto_a_contratar
secop_1_covid.loc[lambda x: x.detalle_del_objeto_a_contratar.str.lower().str.contains('guantes')].shape

In [ ]:
secop_1_covid.groupby('regimen_de_contratacion').agg(n=('regimen_de_contratacion', 'count'))

In [ ]:
secop_1_covid

In [ ]:
secop_2_covid.loc[lambda x: x.proceso_de_compra=='CO1.BDOS.1000964'].iloc[0].url